# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,punta arenas,-53.1500,-70.9167,11.06,54,75,16.98,CL,1733158399
1,1,west island,-12.1568,96.8225,27.99,94,100,2.57,CC,1733158400
2,2,dampit,-8.2116,112.7493,22.48,98,100,0.55,ID,1733158402
3,3,port shepstone,-30.7414,30.4550,21.10,81,100,2.58,ZA,1733158403
4,4,brokopondo,4.7500,-55.1667,29.81,62,99,3.72,SR,1733158404


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot, parametes and sizing

map_plot = city_data_df.hvplot.points(
    x='Lng', 
    y='Lat', 
    geo=True, 
    size='Humidity',
    color='City', 
    alpha=0.5,
    tiles='OSM',
    title='City Humidity Map'
).opts(
    width=1100,  
    height=600  
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [12]:
# Narrow down cities that fit criteria and drop any results with null values

ideal_city_df = city_data_df.loc[(city_data_df['Max Temp'] > 20.00) & 
							 (city_data_df['Max Temp'] < 30.00) &
							 (city_data_df['Wind Speed'] < 4.50) &
							 (city_data_df['Cloudiness'] == 0), :]

# Drop any rows with null values

ideal_city_df = ideal_city_df.dropna(how='any')

# Display sample data

ideal_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,7,chipinge,-20.1883,32.6236,20.96,45,0,3.00,ZW,1733158409
39,39,nampula,-15.1165,39.2666,28.97,34,0,2.68,MZ,1733158446
79,79,port elizabeth,-33.9180,25.5701,22.16,77,0,3.09,ZA,1733158493
104,104,medina,24.4686,39.6142,24.24,23,0,2.06,SA,1733158524
124,124,mulchen,-37.7167,-72.2333,20.74,22,0,4.17,CL,1733158469
205,205,karachi,24.9056,67.0822,22.90,40,0,2.06,PK,1733158421
210,210,lazaro cardenas,17.9583,-102.2000,28.20,58,0,1.43,MX,1733158530
259,259,guerrero,17.6667,-100.0000,25.49,29,0,2.42,MX,1733158703
387,387,holualoa,19.6228,-155.9522,20.31,73,0,3.09,US,1733158677
395,395,gebeit,21.0667,36.3167,20.09,54,0,3.37,SD,1733158866


### Step 3: Create a new DataFrame called `hotel_df`.

In [13]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

hotel_df = ideal_city_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API

hotel_df['Hotel Name'] = ""

# Display sample data

hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
7,chipinge,ZW,-20.1883,32.6236,45,
39,nampula,MZ,-15.1165,39.2666,34,
79,port elizabeth,ZA,-33.9180,25.5701,77,
104,medina,SA,24.4686,39.6142,23,
124,mulchen,CL,-37.7167,-72.2333,22,
205,karachi,PK,24.9056,67.0822,40,
210,lazaro cardenas,MX,17.9583,-102.2000,58,
259,guerrero,MX,17.6667,-100.0000,29,
387,holualoa,US,19.6228,-155.9522,73,
395,gebeit,SD,21.0667,36.3167,54,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [14]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories' : 'accommodation.hotel',
    'limit' : 1,
    'apiKey' : geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():

    # get latitude, longitude from the DataFrame
    
    citylat = row['Lat']
    citylon = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{citylon},{citylat},{radius}"
    params["bias"] =  f"proximity:{citylon},{citylat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
chipinge - nearest hotel: No hotel found
nampula - nearest hotel: Hotel Lurio
port elizabeth - nearest hotel: Waterford Hotel
medina - nearest hotel: The Oberoi Madina
mulchen - nearest hotel: Hostal LaTerraza
karachi - nearest hotel: Quetta Agha Hotel
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
guerrero - nearest hotel: No hotel found
holualoa - nearest hotel: Kona Hotel
gebeit - nearest hotel: No hotel found
musaffah - nearest hotel: No hotel found
villa dolores - nearest hotel: Plaza Hotel
beira - nearest hotel: Hotel Savoy
coalcoman de vazquez pallares - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
7,chipinge,ZW,-20.1883,32.6236,45,No hotel found
39,nampula,MZ,-15.1165,39.2666,34,Hotel Lurio
79,port elizabeth,ZA,-33.9180,25.5701,77,Waterford Hotel
104,medina,SA,24.4686,39.6142,23,The Oberoi Madina
124,mulchen,CL,-37.7167,-72.2333,22,Hostal LaTerraza
205,karachi,PK,24.9056,67.0822,40,Quetta Agha Hotel
210,lazaro cardenas,MX,17.9583,-102.2000,58,Hotel Sol del Pacífico
259,guerrero,MX,17.6667,-100.0000,29,No hotel found
387,holualoa,US,19.6228,-155.9522,73,Kona Hotel
395,gebeit,SD,21.0667,36.3167,54,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [15]:
%%capture --no-display

# Configure the map plot

ideal_map_plot = hotel_df.hvplot.points(
    x='Lng', 
    y='Lat', 
    geo=True, 
    size='Humidity',
    color='City', 
    alpha=0.5,
    tiles='OSM',
    title='City Humidity Map'
).opts(
    width=1100,  
    height=600  
)

# Display the map

ideal_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)